<a href="https://colab.research.google.com/github/amien1410/amien-scrapers/blob/main/Capterra_Scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json
import time
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [4]:
url = "https://www.capterra.com/search/?query=cloudflare"
headers = {'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7', 'accept-encoding': 'gzip, deflate, br, zstd', 'accept-language': 'en-US,en;q=0.9,id;q=0.8,fa;q=0.7,ar;q=0.6,ms;q=0.5,ja;q=0.4,es;q=0.3', 'cookie': '_sp_ses.3006=*; ELOQUA=GUID=E6FFB71CC6BD42B0BD166B662074C6CD&FPCVISITED=1; device=Desktop; country_code=ID; _capterra2_session=150a0edb8e60d80605ad2a42a9156ce2; _sp_id.3006=e24b8902-bd11-4736-9785-c260cff7712f.1762238586.1.1762238617..e0a7ca33-a255-495e-8679-ab2213e905d7..f3f2d0f0-77c3-452b-92ce-2d4313c10233.1762238587146.2; __cf_bm=Jq0bTR6hFwr1VTwB7HtQ3TuSLo_xtNbiu8nNTAOUxCE-1762238630.8974442-1.0.1.1-v.IkVTntWLDgjmaryUg.Du6P6cl8CZoEWpIy8C.C087h3_qVfiqcExKULifmFgyM5ELD2cSVtqErO_2aOvb161jkpjdxRYEpsTSZEQQVlmklcc9VagvAS4k3jhHMijpO; _dd_s=logs=1&id=107109f5-ebd8-4d4a-891e-f38de8e702e0&created=1762238591627&expire=1762239492276', 'priority': 'u=0, i', 'referer': 'https://www.capterra.com/', 'sec-ch-ua': '"Microsoft Edge";v="141", "Not?A_Brand";v="8", "Chromium";v="141"', 'sec-ch-ua-arch': '"x86"', 'sec-ch-ua-bitness': '"64"', 'sec-ch-ua-full-version': '"141.0.3537.99"', 'sec-ch-ua-full-version-list': '"Microsoft Edge";v="141.0.3537.99", "Not?A_Brand";v="8.0.0.0", "Chromium";v="141.0.7390.123"', 'sec-ch-ua-mobile': '?0', 'sec-ch-ua-model': '""', 'sec-ch-ua-platform': '"Windows"', 'sec-ch-ua-platform-version': '"8.0.0"', 'sec-fetch-dest': 'document', 'sec-fetch-mode': 'navigate', 'sec-fetch-site': 'same-origin', 'sec-fetch-user': '?1', 'upgrade-insecure-requests': '1', 'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/141.0.0.0 Safari/537.36 Edg/141.0.0.0'}

res = requests.get(url, headers=headers)
soup = BeautifulSoup(res.text)
soup

<!DOCTYPE html>
<!--[if lt IE 7]> <html class="no-js ie6 oldie" lang="en-US"> <![endif]--><!--[if IE 7]>    <html class="no-js ie7 oldie" lang="en-US"> <![endif]--><!--[if IE 8]>    <html class="no-js ie8 oldie" lang="en-US"> <![endif]--><!--[if gt IE 8]><!--><html class="no-js" lang="en-US"> <!--<![endif]-->
<head>
<title>Attention Required! | Cloudflare</title>
<meta charset="utf-8"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
<meta content="noindex, nofollow" name="robots"/>
<meta content="width=device-width,initial-scale=1" name="viewport"/>
<link href="/cdn-cgi/styles/cf.errors.css" id="cf_styles-css" rel="stylesheet"/>
<!--[if lt IE 9]><link rel="stylesheet" id='cf_styles-ie-css' href="/cdn-cgi/styles/cf.errors.ie.css" /><![endif]-->
<style>body{margin:0;padding:0}</style>
<!--[if gte IE 10]><!-->
<script>
  if (!navigator.cookieEnabled) {
    window.addEventListener('DOMContentLoaded', function () {

In [6]:
import os
import requests
import time
from typing import Dict, Any, Optional

API_BASE = "https://api.apollo.io/api/v1"
API_KEY = "3E1Uz3fL0Vhw4U1Xz4aW8g"

if not API_KEY:
    raise RuntimeError("Please set your APOLLO_API_KEY as an environment variable.")


def _headers() -> Dict[str, str]:
    return {
        "Content-Type": "application/json",
        "X-Api-Key": API_KEY,
    }


def _post_with_retries(url: str, json_payload: Dict[str, Any], max_retries: int = 3, backoff: float = 1.0):
    """POST with retry and exponential backoff."""
    attempt = 0
    while attempt <= max_retries:
        try:
            resp = requests.post(url, json=json_payload, headers=_headers(), timeout=20)
        except requests.RequestException as e:
            attempt += 1
            if attempt > max_retries:
                raise
            time.sleep(backoff * (2 ** (attempt - 1)))
            continue

        # Retry for rate limit or server errors
        if resp.status_code in (429, 500, 502, 503, 504):
            attempt += 1
            if attempt > max_retries:
                resp.raise_for_status()
            time.sleep(backoff * (2 ** (attempt - 1)))
            continue

        # Return JSON on success
        if 200 <= resp.status_code < 300:
            return resp.json()

        # Other errors (client-side)
        try:
            err = resp.json()
        except Exception:
            err = resp.text
        raise RuntimeError(f"API error {resp.status_code}: {err}")


def enrich_by_name(
    first_name: str,
    last_name: Optional[str] = None,
    company: Optional[str] = None,
    domain: Optional[str] = None,
) -> Dict[str, Any]:
    """
    Enrich contact by name.
    Optionally add 'company' or 'domain' for more accurate results.

    Example:
        enrich_by_name("Elon", "Musk", company="Tesla")
    """
    url = f"{API_BASE}/people/match"

    payload = {"first_name": first_name}

    if last_name:
        payload["last_name"] = last_name
    if company:
        payload["organization_name"] = company
    if domain:
        payload["domain"] = domain

    response = _post_with_retries(url, payload)
    return response


def parse_contact_info(resp: Dict[str, Any]) -> Dict[str, Any]:
    """
    Extract useful fields (email, name, company, title, etc.)
    """
    person = resp.get("person") or resp.get("contact") or resp.get("data") or {}
    org = person.get("organization", {}) if isinstance(person, dict) else {}

    return {
        "full_name": person.get("full_name"),
        "title": person.get("title"),
        "company": org.get("name"),
        "email": person.get("email") or person.get("emails"),
        "linkedin_url": person.get("linkedin_url"),
        "raw": person,
    }


if __name__ == "__main__":
    try:
        # Example query — adjust as needed
        result = enrich_by_name("Elon", "Musk", company="Tesla")
        parsed = parse_contact_info(result)

        print("=== Parsed Result ===")
        print(f"Name: {parsed['full_name']}")
        print(f"Company: {parsed['company']}")
        print(f"Title: {parsed['title']}")
        print(f"Email: {parsed['email']}")
        print(f"LinkedIn: {parsed['linkedin_url']}")
    except Exception as e:
        print("Error:", e)


Error: API error 422: {'error': "You have insufficient credits! <a href='https://app.apollo.io/#/settings/plans/upgrade' aria-onclick='close_alert'>Upgrade your plan</a> to increase your number of lead credits."}
